In [1]:
import numpy as np
import pandas as pd
#pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
from tqdm import tqdm
from itertools import permutations
import seaborn as sns
import matplotlib.pyplot as plt
from Bio.Restriction import BsaI, BsmBI
from Bio import SeqIO
from Bio.Seq import Seq
from primers import primers
import primer3 as pr
import warnings
warnings.filterwarnings("ignore")

### JUMP Assembly

#### Circuits Assembly Planning

In [2]:
parts = pd.read_csv('datasets/jump/parts.csv')
sensors = parts[parts['type']=='sensor'].drop(['type', 'site'], axis=1) \
            .rename(columns={'part_name': 'promoter'}).reset_index(drop=True)
rbs = parts[(parts['type']=='rbs') & (parts['site']=='R')].drop(['type', 'site'], axis=1) \
            .rename(columns={'part_name': 'rbs'}).reset_index(drop=True)
rbs_rep = parts[(parts['type']=='rbs') & (parts['site']=='RN')].drop(['type', 'site'], axis=1) \
            .rename(columns={'part_name': 'rbs'}).reset_index(drop=True)
ecfs = parts[parts['type']=='cds'].drop(['type', 'site'], axis=1) \
            .rename(columns={'part_name': 'cds'}).reset_index(drop=True)
reps = parts[parts['type']=='reporter'].drop(['type', 'site'], axis=1) \
            .rename(columns={'part_name': 'cds'}).reset_index(drop=True)
term = parts[(parts['type']=='terminator') & (parts['site']=='T')].drop(['type', 'site'], axis=1) \
            .rename(columns={'part_name': 'terminator'}).reset_index(drop=True)
term_rep = parts[(parts['type']=='terminator') & (parts['site']=='CT')].drop(['type', 'site'], axis=1) \
            .rename(columns={'part_name': 'terminator'}).reset_index(drop=True)

In [3]:
circuit1 = {
    'name': 'circuit1',
    'sensors': ['P{}'.format(i) for i in range(4)],
    'connections': ['W0', 'W1'],
    'gates': [
        'AND({},{},{})'.format('W0', 'P0', 'P1'),
        'AND({},{},{})'.format('W1', 'P2', 'P3'),
        'AND({},{},{})'.format('R0', 'W0', 'W1')
    ]
}
circuit2 = {
    'name': 'circuit2',
    'sensors': ['P{}'.format(i) for i in range(3)],
    'connections': ['W0'],
    'gates': [
        'AND({},{},{})'.format('W0', 'P0', 'P1'),
        'AND({},{},{})'.format('R0', 'P2', 'W0')
    ]
}
circuit3 = {
    'name': 'circuit3',
    'sensors': ['P{}'.format(i) for i in range(2)],
    'connections': [],
    'gates': [
        'AND({},{},{})'.format('R0', 'P0', 'P1')
    ]
}

In [4]:
CONFIG = {
    'single_plasmid': True, #2-plasmids system if False
    'permute_promoter': False,
    'odd_vectors': {
        2: ['1A', '1B*'],
        3: ['1A', '1B', '1C*'],
        4: ['1A', '1B', '1C', '1D'],
        5: ['1A', '1B', '1C', '1D0', '1E'],
        6: ['1A0', '1A1', '1B', '1C', '1D0', '1E'],
        7: ['1A0', '1A1', '1B', '1C', '1D0', '1E0', '1F'],
    },
    'even_vectors': {
        2: ['2A', '2B*'],
        3: ['2A', '2B', '2C*'],
        4: ['2A', '2B', '2C', '2D'],
        5: ['2A', '2B', '2C', '2D0', '2E']
    }
}

#### Gates in Isolation

In [22]:
selected_vectors = CONFIG['odd_vectors'][3] if CONFIG['single_plasmid'] else ['1A-p15a'] + CONFIG['odd_vectors'][2]
selected_vectors

['1A', '1B', '1C*']

In [25]:
#whether to permute promoters or not
if CONFIG['permute_promoter']:
    gates = sensors.merge(sensors, how='cross').merge(ecfs, how='cross').rename(columns={'promoter_x': 'promoter_n', 'promoter_y': 'promoter_c'})
    gates = gates[gates['promoter_n']!=gates['promoter_c']].reset_index(drop=True)
else:
    gates = ecfs.copy()
    gates['promoter_n'] = sensors.iat[0, 0]
    gates['promoter_c'] = sensors.iat[1, 0]

#splitting ECF
gates.rename(columns={'cds': 'ecf_n'}, inplace=True) 
gates['ecf_c'] = gates['ecf_n'] + '_C'
gates['ecf_n'] = gates['ecf_n'] + '_N'

#output promoter
gates['promoter_r'] = pd.DataFrame('P' + gates['ecf_n'].str.split('-', expand=True)[0].str[1:])
reporter = pd.DataFrame(gates['promoter_r']).merge(reps, how='cross') \
           .rename(columns={0: 'promoter_r', 'cds': 'reporter'})

if CONFIG['single_plasmid']:
    gates = pd.merge(reporter, gates, on='promoter_r', how='inner')
    gates = gates[['promoter_r', 'reporter', 'promoter_n', 'promoter_c', 'ecf_n', 'ecf_c']]
else:
    gates = gates[['promoter_n', 'promoter_c', 'ecf_n', 'ecf_c']]
gates['vector'] = '2A'
gates = gates.drop_duplicates().reset_index(drop=True)
gates

,promoter_r,reporter,promoter_n,promoter_c,ecf_n,ecf_c,vector
0,P11,sGFP,PBAD,PCin,E11-PhoRadA_N,E11-PhoRadA_C,2A
1,P11,sGFP,PBAD,PSalTTC,E11-PhoRadA_N,E11-PhoRadA_C,2A
2,P11,sGFP,PBAD,PLuxB,E11-PhoRadA_N,E11-PhoRadA_C,2A
3,P11,sGFP,PCin,PBAD,E11-PhoRadA_N,E11-PhoRadA_C,2A
4,P11,sGFP,PCin,PSalTTC,E11-PhoRadA_N,E11-PhoRadA_C,2A
5,P11,sGFP,PCin,PLuxB,E11-PhoRadA_N,E11-PhoRadA_C,2A
6,P11,sGFP,PSalTTC,PBAD,E11-PhoRadA_N,E11-PhoRadA_C,2A
7,P11,sGFP,PSalTTC,PCin,E11-PhoRadA_N,E11-PhoRadA_C,2A
8,P11,sGFP,PSalTTC,PLuxB,E11-PhoRadA_N,E11-PhoRadA_C,2A
9,P11,sGFP,PLuxB,PBAD,E11-PhoRadA_N,E11-PhoRadA_C,2A


In [12]:
counter = 1
reporter = reporter.merge(rbs_rep, how='cross').merge(term_rep, how='cross') \
           .rename(columns={'promoter_r': 'promoter', 'reporter': 'cds'})
reporter['vector'] = selected_vectors[0]
n_term = gates[['promoter_n', 'ecf_n']].rename(columns={'promoter_n': 'promoter', 'ecf_n': 'cds'})
n_term['vector'] = selected_vectors[1]
c_term = gates[['promoter_c', 'ecf_c']].rename(columns={'promoter_c': 'promoter', 'ecf_c': 'cds'})
c_term['vector'] = selected_vectors[2]

level_1 = reporter.append(n_term.append(c_term).merge(rbs, how='cross').merge(term, how='cross'))
level_1 = level_1[['promoter', 'rbs', 'cds', 'terminator', 'vector']].drop_duplicates().reset_index(drop=True)
level_1['name'] = level_1['promoter'] + '-' + level_1['rbs'].str[-2:] + '-' + \
                  level_1['cds'] + '_' + level_1['vector'].str[1:]
level_1['_id'] = 'A' +  (level_1.index.astype(int) + counter).astype(str)
level_1['component'] = 'gate'
level_1

,promoter,rbs,cds,terminator,vector,name,_id,component
0,P11,RBS34,sGFP,B0015,1A,P11-34-sGFP_A,A1,gate
1,P11,RBS34,mCherry,B0015,1A,P11-34-mCherry_A,A2,gate
2,P20,RBS34,sGFP,B0015,1A,P20-34-sGFP_A,A3,gate
3,P20,RBS34,mCherry,B0015,1A,P20-34-mCherry_A,A4,gate
4,P38,RBS34,sGFP,B0015,1A,P38-34-sGFP_A,A5,gate
5,P38,RBS34,mCherry,B0015,1A,P38-34-mCherry_A,A6,gate
6,PBAD,RBS33,E11-PhoRadA_N,B0015,1B,PBAD-33-E11-PhoRadA_N_B,A7,gate
7,PBAD,RBS33,E20-gp411_N,B0015,1B,PBAD-33-E20-gp411_N_B,A8,gate
8,PBAD,RBS33,E38-gp418_N,B0015,1B,PBAD-33-E38-gp418_N_B,A9,gate
9,PCin,RBS33,E11-PhoRadA_C,B0015,1C*,PCin-33-E11-PhoRadA_C_C*,A10,gate


In [13]:
counter = 1
level_2 = gates.copy()
if CONFIG['single_plasmid']:
    level_2 = pd.merge(level_2, level_1[['promoter', 'cds', '_id']], left_on=['promoter_r', 'reporter'], right_on=['promoter', 'cds'], how='left') \
              .rename(columns={'_id': '_id_r'})
level_2 = pd.merge(level_2, level_1[['promoter', 'cds', '_id']], left_on=['promoter_n', 'ecf_n'], right_on=['promoter', 'cds'], how='left') \
          .rename(columns={'_id': '_id_n'})
level_2 = pd.merge(level_2, level_1[['promoter', 'cds', '_id']], left_on=['promoter_c', 'ecf_c'], right_on=['promoter', 'cds'], how='left') \
          .rename(columns={'_id': '_id_c'})
cols = gates.columns.tolist() + [col for col in level_2.columns if col.startswith('_id')]
level_2 = level_2[cols].drop_duplicates().reset_index(drop=True)   
level_2['name'] = 'pJ2-' + level_2['promoter_n'] + '-' + level_2['promoter_c'] + '-' + level_2['ecf_n'].str[:3]
level_2['_id'] = 'B' + (level_2.index.astype(int) + counter).astype(str)
level_2['component'] = 'circuit'
level_2

,promoter_r,reporter,promoter_n,promoter_c,ecf_n,ecf_c,vector,_id_r,_id_n,_id_c,name,_id,component
0,P11,sGFP,PBAD,PCin,E11-PhoRadA_N,E11-PhoRadA_C,2A,A1,A7,A10,pJ2-PBAD-PCin-E11,B1,circuit
1,P11,mCherry,PBAD,PCin,E11-PhoRadA_N,E11-PhoRadA_C,2A,A2,A7,A10,pJ2-PBAD-PCin-E11,B2,circuit
2,P20,sGFP,PBAD,PCin,E20-gp411_N,E20-gp411_C,2A,A3,A8,A11,pJ2-PBAD-PCin-E20,B3,circuit
3,P20,mCherry,PBAD,PCin,E20-gp411_N,E20-gp411_C,2A,A4,A8,A11,pJ2-PBAD-PCin-E20,B4,circuit
4,P38,sGFP,PBAD,PCin,E38-gp418_N,E38-gp418_C,2A,A5,A9,A12,pJ2-PBAD-PCin-E38,B5,circuit
5,P38,mCherry,PBAD,PCin,E38-gp418_N,E38-gp418_C,2A,A6,A9,A12,pJ2-PBAD-PCin-E38,B6,circuit


#### Gates in Circuit

In [8]:
CONFIG = {
    'single_plasmid': True, #2-plasmids system if False
    'permute_promoter': False,
    'odd_vectors': {
        2: ['1A', '1B*'],
        3: ['1A', '1B', '1C*'],
        4: ['1A', '1B', '1C', '1D'],
        5: ['1A', '1B', '1C', '1D0', '1E'],
        6: ['1A0', '1A1', '1B', '1C', '1D0', '1E'],
        7: ['1A0', '1A1', '1B', '1C', '1D0', '1E0', '1F'],
    },
    'even_vectors': {
        2: ['2A', '2B*'],
        3: ['2A', '2B', '2C*'],
        4: ['2A', '2B', '2C', '2D'],
        5: ['2A', '2B', '2C', '2D0', '2E']
    }
}

circuit_design = circuit3.copy()

In [9]:
num_tus = len(circuit_design['gates']) * 2 + 1 if CONFIG['single_plasmid'] else len(circuit_design['gates']) * 2
selected_vectors = CONFIG['odd_vectors'][num_tus]
selected_vectors

['1A', '1B', '1C*']

In [10]:
prom = None
num_inputs = len(circuit_design['gates']) + 1
if CONFIG['permute_promoter']:
    prom = pd.DataFrame(list(permutations(sensors['promoter'].tolist(), num_inputs)))   
else:
    prom = sensors.T.reset_index(drop=True).iloc[:,:num_inputs]
prom.columns = ['P{}'.format(n) for n in range(num_inputs)]
for n in range(num_inputs):
    prom['P{}*'.format(n)] = 'P{}'.format(n)
prom

,P0,P1,P0*,P1*
0,PBAD,PCin,P0,P1


In [11]:
def generate_arrangement(circuit, vectors):
    
    arr = pd.DataFrame(list(permutations(ecfs['cds'].tolist(), len(circuit['gates']))))
    if len(arr)==0:
        print('Not enough number of gates!')
    else:
        
        inputs = []
        for gate in circuit['gates']:
            inputs.extend(gate.split('(')[1].replace(')', '').split(',')[1:])
        
        for i, inp in enumerate(inputs):
            if inp.startswith('P'):
                arr[(inp) + '*'] = inp
            elif inp.startswith('W'):
                arr['P{}'.format(i)] = 'P' + arr[int(inp[-1])].str[1:3]
            
        if len(vectors) < len(circuit['gates']):
            print('Not enough vectors!')
            return None, None
        else:
            for i in range(len(circuit['gates'])):
                arr['vector_n_{}'.format(i)] = vectors[i][0]
                arr['vector_c_{}'.format(i)] = vectors[i][1]
            return arr, inputs

        
keys = ['P{}*'.format(n) for n in range(num_inputs)]
vector_iter = iter(selected_vectors[1:]) if CONFIG['single_plasmid'] else iter(selected_vectors)
vector_list = list(zip(vector_iter, vector_iter)) #group vectors into a tuple of two vectors
circuit, inputs = generate_arrangement(circuit_design, vector_list)
circuit = pd.merge(circuit, prom, on=keys, how='left').drop(keys, axis=1)
circuit

,0,vector_n_0,vector_c_0,P0,P1
0,E11-PhoRadA,1B,1C*,PBAD,PCin
1,E20-gp411,1B,1C*,PBAD,PCin
2,E38-gp418,1B,1C*,PBAD,PCin


In [12]:
gates = circuit.copy()

In [13]:
reporter = reporter.merge(rbs_rep, how='cross').merge(term_rep, how='cross') \
           .rename(columns={'promoter_r': 'promoter', 'reporter': 'cds'})
reporter['vector'] = selected_vectors[0]
reporter

NameError: name 'reporter' is not defined

In [ ]:
reporter = reporter.merge(rbs_rep, how='cross').merge(term_rep, how='cross') \
           .rename(columns={'promoter_r': 'promoter', 'reporter': 'cds'})
reporter['vector'] = selected_vectors[0]
n_term = gates[['promoter_n', 'ecf_n']].rename(columns={'promoter_n': 'promoter', 'ecf_n': 'cds'})
n_term['vector'] = selected_vectors[1]
c_term = gates[['promoter_c', 'ecf_c']].rename(columns={'promoter_c': 'promoter', 'ecf_c': 'cds'})
c_term['vector'] = selected_vectors[2]

level_1 = reporter.append(n_term.append(c_term).merge(rbs, how='cross').merge(term, how='cross'))
level_1 = level_1[['promoter', 'rbs', 'cds', 'terminator', 'vector']].drop_duplicates().reset_index(drop=True)
level_1['name'] = level_1['promoter'] + '-' + level_1['rbs'].str[-2:] + '-' + \
                  level_1['cds'] + '_' + level_1['vector'].str[1:]
level_1['_id'] = 'A' +  (level_1.index.astype(int) + counter).astype(str)
level_1['component'] = 'gate'
level_1

In [14]:
components = pd.DataFrame()
num_gates = len(circuit_design['gates'])
gates = circuit[[i for i in range(num_gates)]]
inputs = circuit[['P{}'.format(i) for i in range(num_gates*2)]]
vectors = circuit[[col for col in circuit.columns.tolist() if str(col).startswith('vector')]]
for n in range(num_gates):
    temp = pd.concat([inputs.iloc[:,n*2], inputs.iloc[:,n*2+1], gates.iloc[:,n], vectors.iloc[:,n*2], vectors.iloc[:,n*2+1]], axis=1)
    temp.columns = ['promoter_n', 'promoter_c', 'ecf_n', 'vector_n', 'vector_c']
    components = pd.concat([components, temp], axis=0)
components = components.drop_duplicates().reset_index(drop=True)
components['ecf_c'] = components['ecf_n'] + '_C'
components['ecf_n'] = components['ecf_n'] + '_N'
components

,promoter_n,promoter_c,ecf_n,vector_n,vector_c,ecf_c
0,PBAD,PCin,E11-PhoRadA_N,1B,1C*,E11-PhoRadA_C
1,PBAD,PCin,E20-gp411_N,1B,1C*,E20-gp411_C
2,PBAD,PCin,E38-gp418_N,1B,1C*,E38-gp418_C


In [15]:
output_prom = pd.DataFrame('P' + ecfs['part_name'].str.split('-', expand=True)[0].str[1:])
output_gate = output_prom.merge(rbs_rep, how='cross').merge(reps, how='cross').merge(term_rep, how='cross')
output_gate.columns = ['promoter', 'rbs', 'cds', 'terminator']
output_gate['name'] = output_gate['promoter'] + '-' + output_gate['rbs'].str[-2:] + '-' + \
                           output_gate['cds']
output_gate = output_gate[['name', 'promoter', 'rbs', 'cds', 'terminator']]
if CONFIG['num_plasmids']==2:
    output_gate['vector'] = '1A-p15a'
else:
    output_gate['vector'] = selected_vectors[0]
output_gate['name'] = output_gate['name'] + '_' + output_gate['vector'].str[1:]
output_gate['component'] = 'circuit'
output_gate

KeyError: 'part_name'

In [ ]:
level_2 = components.merge(rbs, how='cross').merge(term, how='cross') \
                .merge(rbs, how='cross').merge(term, how='cross') \
                .drop_duplicates().reset_index(drop=True)
level_2.columns = ['promoter_n', 'promoter_c', 'ecf_n', 'vector_n', 'vector_c', 'ecf_c', \
                   'rbs_n', 'terminator_n', 'rbs_c', 'terminator_c']
level_2 = level_2.drop_duplicates().reset_index(drop=True)
level_2['name'] = 'pJ2-' + level_2['promoter_n'] + '-' + level_2['promoter_c'] + '-' + level_2['ecf_n'].str[:3]
level_2['name_n'] = 'pJ1-' + level_2['promoter_n'] + '-' + level_2['ecf_n'].str[:3] + '-N' + '_' + level_2['vector_n'].str[1:]
level_2['name_c'] = 'pJ1-' + level_2['promoter_c'] + '-' + level_2['ecf_c'].str[:3] + '-C' + '_' + level_2['vector_c'].str[1:]
level_2['vector'] = '2A'
gate_level_2 = level_2[['name', 'name_n', 'name_c', 'vector',
                   'promoter_n', 'rbs_n', 'ecf_n', 'terminator_n', 'vector_n',
                   'promoter_c', 'rbs_c', 'ecf_c', 'terminator_c', 'vector_c']]
gate_level_2['promoter'] = 'P' + gate_level_2['ecf_n'].str[1:3]
gate_level_2 = pd.merge(gate_level_2, output_gate, on='promoter', how='left')
gate_level_2.rename(columns={'name_x': 'name', 'name_y': 'name_r', 'vector_x': 'vector'}, inplace=True)
#gate_level_2

In [ ]:
n_terms = gate_level_2[['name_n', 'promoter_n', 'rbs_n', 'ecf_n', 'terminator_n', 'vector_n', 'component']]
n_terms.columns = ['name', 'promoter', 'rbs', 'cds', 'terminator', 'vector', 'component']
c_terms = gate_level_2[['name_c', 'promoter_c', 'rbs_c', 'ecf_c', 'terminator_c', 'vector_c', 'component']]
c_terms.columns = ['name', 'promoter', 'rbs', 'cds', 'terminator', 'vector', 'component']
level_1 = n_terms.append(c_terms).drop_duplicates()
start = 0
gate_all_lvl1 = output_gate.append(level_1).reset_index(drop=True)
gate_all_lvl1['_id'] = pd.Series(gate_all_lvl1.index).apply(lambda x: 'C' + str(x+1+start).zfill(3))
gate_all_lvl1[['_id', 'name', 'promoter', 'rbs', 'cds', 'terminator', 'vector', 'component']]

In [ ]:
gate_all_lvl1[['_id', 'name', 'promoter', 'rbs', 'cds', 'terminator', 'vector', 'component']].to_csv('datasets/jump/circuit-level-1.csv', index=False)

In [ ]:
for n in range(num_gates):
    arr2['ecf_n_{}'.format(n)] = arr2[n] + '_N'
    arr2['ecf_c_{}'.format(n)] = arr2[n] + '_C'
arr2 = arr2.drop([n for n in range(num_gates)], axis=1)
arr2

In [ ]:
start = len(gate_all_lvl1)
gate_level_2_id = arr2.copy()
for n in range(num_gates):
    gate_level_2_id = pd.merge(gate_level_2_id, gate_all_lvl1[['_id', 'cds']],
            left_on='ecf_n_{}'.format(n), right_on='cds', how='left') \
            .rename(columns={'_id': '_id_n_{}'.format(n)})
    gate_level_2_id = pd.merge(gate_level_2_id, gate_all_lvl1[['_id', 'cds']],
            left_on='ecf_c_{}'.format(n), right_on='cds', how='left') \
            .rename(columns={'_id': '_id_c_{}'.format(n)})
cols = np.array([['_id_c_{}'.format(n), 'ecf_n_{}'.format(n), 'ecf_c_{}'.format(n)] for n in range(num_gates)]).ravel().tolist()
gate_level_2_id = gate_level_2_id[cols]

for n in range(num_gates):    
    gate_level_2_id = pd.merge(gate_level_2_id, gate_all_lvl1[['_id', 'cds']],
            left_on='ecf_n_{}'.format(n), right_on='cds', how='left') \
            .rename(columns={'_id': '_id_n_{}'.format(n)})
cols = np.array([['_id_n_{}'.format(n), 'ecf_n_{}'.format(n), '_id_c_{}'.format(n), 'ecf_c_{}'.format(n)] for n in range(num_gates)]).ravel().tolist()
gate_level_2_id = gate_level_2_id[cols]
gate_level_2_id

In [ ]:
n_terms = circuit_level_2[['name_n', 'promoter_n', 'rbs_n', 'ecf_n', 'terminator_n', 'vector_n']]
n_terms.columns = ['name', 'promoter', 'rbs', 'cds', 'terminator', 'vector']
c_terms = circuit_level_2[['name_c', 'promoter_c', 'rbs_c', 'ecf_c', 'terminator_c', 'vector_c']]
c_terms.columns = ['name', 'promoter', 'rbs', 'cds', 'terminator', 'vector']
level_1 = n_terms.append(c_terms).drop_duplicates().reset_index(drop=True)
level_1['component'] = 'circuit'
level_1

In [ ]:
circuit_all_lvl1 = output_gate.append(level_1).reset_index(drop=True)
circuit_all_lvl1

In [ ]:
all_lvl1 = gate_all_lvl1.append(circuit_all_lvl1).drop_duplicates().reset_index(drop=True)
all_lvl1['_id'] = pd.Series(all_lvl1.index).apply(lambda x: 'G' + str(x+1).zfill(3))
all_lvl1 = all_lvl1[['_id', 'name', 'promoter', 'rbs', 'cds', 'terminator', 'vector', 'component']]
all_lvl1

In [ ]:
all_lvl2 = gate_level_2.append(circuit_level_2).drop_duplicates().reset_index(drop=True) \
            [['name', 'name_n', 'name_c', 'vector', 'component']]
all_lvl2 = pd.merge(all_lvl2, all_lvl1, left_on='name_n', right_on='name', how='left') \
           [['name_x', '_id', 'name_n', 'name_c', 'vector_x', 'component_x']]
all_lvl2.columns = ['name', '_id_n', 'name_n', 'name_c', 'vector', 'component']
all_lvl2 = pd.merge(all_lvl2, all_lvl1, left_on='name_c', right_on='name', how='left') \
            [['name_x', '_id_n', 'name_n', '_id', 'name_c', 'vector_x', 'component_x']]
all_lvl2.columns = ['name', '_id_n', 'name_n', '_id_c', 'name_c', 'vector', 'component']
all_lvl2['_id'] = pd.Series(all_lvl2.index).apply(lambda x: 'G' + str(x+len(all_lvl1)+1).zfill(3))
all_lvl2 = all_lvl2[['_id', 'name', '_id_n', 'name_n', '_id_c', 'name_c', 'vector', 'component']]
all_lvl2